<a href="https://colab.research.google.com/github/Gaukhar-ai/working-in-pairs/blob/master/Starbucks_distance_IL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [28]:
#!pip install haversine

In [30]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [32]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [5]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv')
df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [43]:
starbucks = df.loc[(df['State/Province'] =='IL') & 
                   (df['Country'] == 'US'), :].copy()
starbucks.head()


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17457,Starbucks,2726-85202,Lake & Rohlwing,Company Owned,"1590 W. Lake St., Suite 101",Addison,IL,US,601011824,630773-4743,GMT-06:00 America/Chicago,-88.03,41.95,"(41.95, -88.03)"
17458,Starbucks,76107-79323,Super Target Algonquin ST-1801,Licensed,"750 Randall Rd, Ryan Place",Algonquin,IL,US,60102,847-458-5340,GMT-06:00 America/Chicago,-88.34,42.17,"(42.17, -88.34)"
17459,Starbucks,2626-63429,East Algonquin & Ryan,Company Owned,"1532 E. Algonquin Road, Floor 1",Algonquin,IL,US,601024519,847-458-6740,GMT-06:00 America/Chicago,-88.27,42.16,"(42.16, -88.27)"
17460,Starbucks,18024-182807,Randall & Harnish,Company Owned,"790 S. Randall Road, Suite 1",Algonquin,IL,US,60102,8474580669,GMT-06:00 America/Chicago,-88.34,42.16,"(42.16, -88.34)"
17461,Starbucks,21358-211207,Arlington Heights & Golf Road,Company Owned,1808 South Arlington Heights Road,Arlington Heights,IL,US,60005,8476402779,GMT-06:00 America/Chicago,-87.98,42.05,"(42.05, -87.98)"


The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
39.14, -121.64  
34.13, -116.40  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
39.14, -121.64,   (39.14, -121.64)  
34.13, -116.40,   (34.13, -116.40)  
...

In [45]:
# answer goes here
starbucks['Coordinates'] = list(zip(starbucks['Latitude'],
                                    starbucks['Longitude']))
starbucks.head()


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17457,Starbucks,2726-85202,Lake & Rohlwing,Company Owned,"1590 W. Lake St., Suite 101",Addison,IL,US,601011824,630773-4743,GMT-06:00 America/Chicago,-88.03,41.95,"(41.95, -88.03)"
17458,Starbucks,76107-79323,Super Target Algonquin ST-1801,Licensed,"750 Randall Rd, Ryan Place",Algonquin,IL,US,60102,847-458-5340,GMT-06:00 America/Chicago,-88.34,42.17,"(42.17, -88.34)"
17459,Starbucks,2626-63429,East Algonquin & Ryan,Company Owned,"1532 E. Algonquin Road, Floor 1",Algonquin,IL,US,601024519,847-458-6740,GMT-06:00 America/Chicago,-88.27,42.16,"(42.16, -88.27)"
17460,Starbucks,18024-182807,Randall & Harnish,Company Owned,"790 S. Randall Road, Suite 1",Algonquin,IL,US,60102,8474580669,GMT-06:00 America/Chicago,-88.34,42.16,"(42.16, -88.34)"
17461,Starbucks,21358-211207,Arlington Heights & Golf Road,Company Owned,1808 South Arlington Heights Road,Arlington Heights,IL,US,60005,8476402779,GMT-06:00 America/Chicago,-87.98,42.05,"(42.05, -87.98)"


In [ ]:
# answer goes here
#import ast

#def convert(Coordinates):
    #return ast.literal_eval(d)

#convert(" ")
           
#df_IL['Country'].nunique()




Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [46]:
# answer goes here
distance_matrix = geo_sim_matrix(starbucks)
distance_matrix.head()


index,17457,17458,17459,17460,17461,17462,17463,17464,17465,17466,17467,17468,17469,17470,17471,17472,17473,17474,17475,17476,17477,17478,17479,17480,17481,17482,17483,17484,17485,17486,17487,17488,17489,17490,17491,17492,17493,17494,17495,17496,...,17992,17993,17994,17995,17996,17997,17998,17999,18000,18001,18002,18003,18004,18005,18006,18007,18008,18009,18010,18011,18012,18013,18014,18015,18016,18017,18018,18019,18020,18021,18022,18023,18024,18025,18026,18027,18028,18029,18030,18031
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17457,0.000000,21.998655,19.029597,21.527960,7.370898,9.341678,9.686688,11.348866,21.130444,18.405280,16.120442,19.890939,16.092477,16.066980,17.335603,15.960145,14.740388,14.926717,9.249612,17.375211,19.426136,257.085277,259.596589,45.816298,45.188039,4.827536,13.698377,2.168336,4.676492,111.006613,112.258420,113.955064,17.884110,17.395697,18.687674,18.081975,15.899822,55.061650,54.690383,54.690383,...,127.440211,20.355958,20.271849,20.452576,17.879419,21.808011,3.774106,6.728997,29.221818,12.621019,12.621019,32.953274,15.708955,30.018943,28.428320,9.656461,16.896644,9.614315,11.759781,10.009218,9.039216,5.839672,6.890137,8.406242,8.409842,14.499963,14.926717,14.569752,13.920860,18.722283,16.516217,16.516217,19.506333,18.539901,13.856968,12.606001,12.479327,32.891659,29.349573,29.109865
17458,21.998655,0.000000,3.650891,0.690934,20.229834,19.468171,17.303818,18.904129,26.335757,30.067912,25.816109,25.585193,25.063751,29.104932,28.703156,27.431645,10.843998,10.336209,16.599056,22.109893,41.398377,265.547496,267.723281,25.925913,25.038253,26.384532,35.459646,20.058238,19.491601,119.646125,120.980590,122.588754,34.386293,35.314882,34.877324,35.951677,34.704488,73.547993,73.575935,73.575935,...,142.518683,20.040996,19.544708,20.538045,20.583364,20.903626,25.767579,28.289432,15.744719,27.014358,27.014358,29.612019,28.087659,27.278014,25.708294,19.162867,5.473141,31.608526,33.604592,30.991694,26.527741,22.662958,23.854364,23.752460,25.917411,22.122397,21.555503,20.533378,34.996893,32.877956,30.365815,30.365815,33.256965,31.018190,33.609257,30.922485,31.396745,11.190566,34.509039,35.610755
17459,19.029597,3.650891,0.000000,3.585198,16.696038,15.856924,13.693897,15.255140,26.181536,29.037429,24.873630,25.299526,24.188151,27.808760,27.656463,26.300691,7.204171,6.694501,14.832526,21.718386,38.307869,266.073124,268.308339,29.575682,28.689060,23.221572,32.265566,17.210895,17.037709,119.997495,121.320535,122.947491,32.704848,33.404274,33.261840,34.062580,32.618340,71.823649,71.755307,71.755307,...,141.688324,16.802458,16.319970,17.286636,17.120833,17.774387,22.800204,25.520682,14.734481,25.398393,25.398393,27.158454,24.437615,24.661508,23.029821,17.545641,4.270400,28.629394,30.755475,28.401243,24.414801,20.382764,21.655160,21.775969,23.770575,18.492762,17.940703,16.917087,32.423611,29.227088,26.715266,26.715266,29.607542,27.374610,31.320382,28.838915,29.220768,13.863712,34.722152,35.635412
17460,21.527960,0.690934,3.585198,0.000000,19.957956,19.260005,17.096893,18.766155,25.647000,29.385448,25.132133,24.894833,24.378238,28.433001,28.021526,26.753324,10.778609,10.267503,15.968966,21.418960,40.942171,264.893414,267.072992,26.047315,25.182740,25.957131,35.034729,19.565026,18.933041,118.980589,120.314495,121.923664,33.734849,34.681033,34.221251,35.315738,34.087407,72.893323,72.928172,72.928172,...,141.828686,20.220341,19.728487,20.713164,20.665985,21.120960,25.291797,27.770938,16.325634,26.361677,26.361677,30.005985,27.979126,27.636037,26.051103,18.516900,4.873422,31.130433,33.097651,30.440831,25.918331,22.076576,23.256969,23.131694,25.312165,22.070119,21.523956,20.500091,34.440970,32.756815,30.234197,30.234197,33.151695,30.935839,33.015211,30.307037,30.790857,11.793137,33.831552,34.927262
17461,7.370898,20.229834,16.696038,19.957956,0.000000,2.072803,3.441989,4.145605,27.305780,25.526491,22.817007,26.076186,22.644605,23.267332,24.377342,22.950306,10.337925,10.724738,13.682370

For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [48]:
# answer goes here

# Replace diag values with NaN 
# A store shouldn't be considered closest to itself
np.fill_diagonal(distance_matrix.values, np.nan)

# Find index with min value for each store
starbucks['Nearest Neighbor'] = distance_matrix.idxmin()
# Find min value for each store
starbucks['Distance (Miles)'] = distance_matrix.min()

starbucks.head()




,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,Nearest Neighbor,Distance (Miles)
17457,Starbucks,2726-85202,Lake & Rohlwing,Company Owned,"1590 W. Lake St., Suite 101",Addison,IL,US,601011824,630773-4743,GMT-06:00 America/Chicago,-88.03,41.95,"(41.95, -88.03)",17797,1.722053
17458,Starbucks,76107-79323,Super Target Algonquin ST-1801,Licensed,"750 Randall Rd, Ryan Place",Algonquin,IL,US,60102,847-458-5340,GMT-06:00 America/Chicago,-88.34,42.17,"(42.17, -88.34)",17460,0.690934
17459,Starbucks,2626-63429,East Algonquin & Ryan,Company Owned,"1532 E. Algonquin Road, Floor 1",Algonquin,IL,US,601024519,847-458-6740,GMT-06:00 America/Chicago,-88.27,42.16,"(42.16, -88.27)",17807,3.368986
17460,Starbucks,18024-182807,Randall & Harnish,Company Owned,"790 S. Randall Road, Suite 1",Algonquin,IL,US,60102,8474580669,GMT-06:00 America/Chicago,-88.34,42.16,"(42.16, -88.34)",17458,0.690934
17461,Starbucks,21358-211207,Arlington Heights & Golf Road,Company Owned,1808 South Arlington Heights Road,Arlington Heights,IL,US,60005,8476402779,GMT-06:00 America/Chicago,-87.98,42.05,"(42.05, -87.98)",17934,1.026121


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [52]:
# answer goes here
sorted_starbucks = starbucks.sort_values('Distance (Miles)', ascending=False)
sorted_starbucks.head()




,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,Nearest Neighbor,Distance (Miles)
17845,Starbucks,48900-254804,Kroger Mt. Vernon 711 - Venue,Licensed,415 SOUTH 42ND STREET,MT. VERNON,IL,US,62864,618-244-6210,GMT-06:00 America/Chicago,-88.94,38.31,"(38.31, -88.94)",17828,38.753508
17827,Starbucks,26714-244394,Hy-Vee Macomb #1420,Licensed,1602 East Jackson,Macomb,IL,US,61455,3098379917,GMT-06:00 America/Chicago,-90.65,40.46,"(40.46, -90.65)",17505,32.930632
17515,Starbucks,13613-106969,Charleston-Lincoln & E St,Company Owned,437 W Lincoln Avenue,Charleston,IL,US,619203022,217 345 5553,GMT-06:00 America/Chicago,-88.19,39.48,"(39.48, -88.19)",17728,31.057108
17728,Starbucks,9648-96552,I-70 & Hwy 32,Company Owned,1700 Avenue of Mid America,Effingham,IL,US,624014500,217-342-5577,GMT-06:00 America/Chicago,-88.57,39.14,"(39.14, -88.57)",17515,31.057108
17754,Starbucks,24096-231765,Hy-Vee @ Galesburg #2,Licensed,"2030 E Main St, Geneva Commons Shopping Center",Galesburg,IL,US,61401,309-342-1615,GMT-06:00 America/Chicago,-90.34,40.95,"(40.95, -90.34)",17505,30.591711
